In [3]:
import pandas as pd
import geopandas as gpd
import re
from shapely.geometry import Point

file_path = '/Users/yujinlee/Google_Drive/NYU/Fall_2023/Capstone/CrimeData'
df = pd.read_csv(file_path + '/seattle_2018_to_2023.csv', low_memory = False)

Crime data: https://data.seattle.gov/Public-Safety/SPD-Crime-Data-2008-Present 
<br>Zipcode data: https://github.com/OpenDataDE/State-zip-code-GeoJSON/blob/master/wa_washington_zip_codes_geo.min.json

In [4]:
df.head()

,Report Number,Offense ID,Offense Start DateTime,Offense End DateTime,Report DateTime,Group A B,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,Beat,MCPP,100 Block Address,Longitude,Latitude
0,2018-347660,7660504950,01/01/2018 12:00:00 AM,09/17/2018 02:08:00 PM,09/17/2018 04:05:00 PM,A,PROPERTY,LARCENY-THEFT,Theft From Building,23D,N,J,J1,BALLARD NORTH,8XX BLOCK OF NW 95TH ST,-122.366706,47.697866
1,2018-306159,7648277122,01/01/2018 12:00:00 AM,08/17/2018 07:34:00 AM,08/17/2018 07:34:00 AM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,S,S,S2,RAINIER BEACH,86XX BLOCK OF ISLAND DR S,-122.261000,47.525617
2,2018-304095,7639729915,01/01/2018 12:00:00 AM,03/31/2018 07:00:00 AM,08/15/2018 08:53:00 PM,A,PERSON,SEX OFFENSES,Fondling,11D,N,L,L2,NORTHGATE,NaN,0.000000,0.000000
3,2018-907128,7697846930,01/01/2018 12:00:00 AM,01/01/2018 12:00:00 AM,08/04/2018 06:35:00 PM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,N,J,J3,PHINNEY RIDGE,7XX BLOCK OF N 70TH ST,-122.348588,47.679599
4,2018-283356,7698871738,01/01/2018 12:00:00 AM,01/31/2018 11:59:00 PM,07/31/2018 07:02:00 PM,A,PERSON,SEX OFFENSES,Rape,11A,N,N,N3,NORTHGATE,NaN,0.000000,0.000000


In [7]:
df.shape

(395801, 17)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395801 entries, 0 to 395800
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Report Number           395801 non-null  object 
 1   Offense ID              395801 non-null  int64  
 2   Offense Start DateTime  395801 non-null  object 
 3   Offense End DateTime    258945 non-null  object 
 4   Report DateTime         395801 non-null  object 
 5   Group A B               395801 non-null  object 
 6   Crime Against Category  395801 non-null  object 
 7   Offense Parent Group    395801 non-null  object 
 8   Offense                 395801 non-null  object 
 9   Offense Code            395801 non-null  object 
 10  Precinct                395792 non-null  object 
 11  Sector                  395794 non-null  object 
 12  Beat                    395794 non-null  object 
 13  MCPP                    395796 non-null  object 
 14  100 Block Address   

In [9]:
df.columns = map(str.lower, df.columns)
df.columns = df.columns.str.replace(' ', '_')
df.head()

,report_number,offense_id,offense_start_datetime,offense_end_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,precinct,sector,beat,mcpp,100_block_address,longitude,latitude
0,2018-347660,7660504950,01/01/2018 12:00:00 AM,09/17/2018 02:08:00 PM,09/17/2018 04:05:00 PM,A,PROPERTY,LARCENY-THEFT,Theft From Building,23D,N,J,J1,BALLARD NORTH,8XX BLOCK OF NW 95TH ST,-122.366706,47.697866
1,2018-306159,7648277122,01/01/2018 12:00:00 AM,08/17/2018 07:34:00 AM,08/17/2018 07:34:00 AM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,S,S,S2,RAINIER BEACH,86XX BLOCK OF ISLAND DR S,-122.261000,47.525617
2,2018-304095,7639729915,01/01/2018 12:00:00 AM,03/31/2018 07:00:00 AM,08/15/2018 08:53:00 PM,A,PERSON,SEX OFFENSES,Fondling,11D,N,L,L2,NORTHGATE,NaN,0.000000,0.000000
3,2018-907128,7697846930,01/01/2018 12:00:00 AM,01/01/2018 12:00:00 AM,08/04/2018 06:35:00 PM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,N,J,J3,PHINNEY RIDGE,7XX BLOCK OF N 70TH ST,-122.348588,47.679599
4,2018-283356,7698871738,01/01/2018 12:00:00 AM,01/31/2018 11:59:00 PM,07/31/2018 07:02:00 PM,A,PERSON,SEX OFFENSES,Rape,11A,N,N,N3,NORTHGATE,NaN,0.000000,0.000000


In [ ]:
df['offense_start_datetime'] = df['offense_start_datetime'].str.slice(0, 10)
# Convert the 'occurred_on_date' column to datetime
df['offense_start_datetime'] = pd.to_datetime(df['offense_start_datetime'], format='%m/%d/%Y')

In [13]:
df.head()

,report_number,offense_id,offense_start_datetime,offense_end_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,precinct,sector,beat,mcpp,100_block_address,longitude,latitude
0,2018-347660,7660504950,2018-01-01,09/17/2018 02:08:00 PM,09/17/2018 04:05:00 PM,A,PROPERTY,LARCENY-THEFT,Theft From Building,23D,N,J,J1,BALLARD NORTH,8XX BLOCK OF NW 95TH ST,-122.366706,47.697866
1,2018-306159,7648277122,2018-01-01,08/17/2018 07:34:00 AM,08/17/2018 07:34:00 AM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,S,S,S2,RAINIER BEACH,86XX BLOCK OF ISLAND DR S,-122.261000,47.525617
2,2018-304095,7639729915,2018-01-01,03/31/2018 07:00:00 AM,08/15/2018 08:53:00 PM,A,PERSON,SEX OFFENSES,Fondling,11D,N,L,L2,NORTHGATE,NaN,0.000000,0.000000
3,2018-907128,7697846930,2018-01-01,01/01/2018 12:00:00 AM,08/04/2018 06:35:00 PM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,N,J,J3,PHINNEY RIDGE,7XX BLOCK OF N 70TH ST,-122.348588,47.679599
4,2018-283356,7698871738,2018-01-01,01/31/2018 11:59:00 PM,07/31/2018 07:02:00 PM,A,PERSON,SEX OFFENSES,Rape,11A,N,N,N3,NORTHGATE,NaN,0.000000,0.000000


In [17]:
df[df['latitude']==0].shape

(23759, 17)

In [18]:
df[df['longitude']==0].shape

(23759, 17)

In [19]:
df = df[df['longitude']!=0]

In [22]:
df['location'] = df.apply(lambda row: f"({row['latitude']}, {row['longitude']})", axis=1)

In [23]:
file_path_geo = file_path + '/wa_washington_zip_codes_geo.min.json'

zipcode = gpd.read_file(file_path_geo)
zipcode.head()

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,53,98822,5398822,B5,G6350,S,1131837710,5582389,+47.9019257,-120.5504512,N,"POLYGON ((-120.47985 47.68373, -120.48008 47.6..."
1,53,98821,5398821,B5,G6350,S,4754899,198324,+47.5497169,-120.5586129,N,"POLYGON ((-120.57188 47.55317, -120.57191 47.5..."
2,53,98357,5398357,B5,G6350,S,110004759,462073,+48.3338551,-124.6354040,N,"MULTIPOLYGON (((-124.74255 48.39175, -124.7424..."
3,53,98663,5398663,B5,G6350,S,11134084,70154,+45.6573955,-122.6631613,N,"POLYGON ((-122.67066 45.64994, -122.67066 45.6..."
4,53,98684,5398684,B5,G6350,S,18064308,7161,+45.6305560,-122.5148386,N,"POLYGON ((-122.50606 45.64292, -122.50584 45.6..."


In [24]:
def create_point_from_string(coord_str):
    match = re.match(r'\(([^,]+), ([^)]+)\)', coord_str)
    if match:
        lat, lon = map(float, match.groups())
        return Point(lon, lat)
    return None

In [25]:
df['geometry'] = df['location'].apply(create_point_from_string)

In [26]:
df.head()

,report_number,offense_id,offense_start_datetime,offense_end_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,precinct,sector,beat,mcpp,100_block_address,longitude,latitude,location,geometry
0,2018-347660,7660504950,2018-01-01,09/17/2018 02:08:00 PM,09/17/2018 04:05:00 PM,A,PROPERTY,LARCENY-THEFT,Theft From Building,23D,N,J,J1,BALLARD NORTH,8XX BLOCK OF NW 95TH ST,-122.366706,47.697866,"(47.69786645, -122.36670644)",POINT (-122.36670644 47.69786645)
1,2018-306159,7648277122,2018-01-01,08/17/2018 07:34:00 AM,08/17/2018 07:34:00 AM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,S,S,S2,RAINIER BEACH,86XX BLOCK OF ISLAND DR S,-122.261000,47.525617,"(47.52561712, -122.260999956)",POINT (-122.260999956 47.52561712)
3,2018-907128,7697846930,2018-01-01,01/01/2018 12:00:00 AM,08/04/2018 06:35:00 PM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,N,J,J3,PHINNEY RIDGE,7XX BLOCK OF N 70TH ST,-122.348588,47.679599,"(47.67959898, -122.348587683)",POINT (-122.348587683 47.67959898)
5,2018-214446,7678773428,2018-01-01,06/13/2018 02:41:00 PM,06/13/2018 02:41:00 PM,A,PROPERTY,FRAUD OFFENSES,False Pretenses/Swindle/Confidence Game,26A,S,O,O2,SODO,8XX BLOCK OF S ADAMS ST,-122.321761,47.566196,"(47.56619631, -122.321760505)",POINT (-122.321760505 47.56619631)
6,2018-137495,7700168935,2018-01-01,04/19/2018 09:53:00 AM,04/19/2018 09:53:00 AM,A,PROPERTY,COUNTERFEITING/FORGERY,Counterfeiting/Forgery,250,S,O,O2,SODO,38XX BLOCK OF 1ST AVE S,-122.334206,47.569889,"(47.56988892, -122.334205833)",POINT (-122.334205833 47.56988892)


In [27]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.head()

,report_number,offense_id,offense_start_datetime,offense_end_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,precinct,sector,beat,mcpp,100_block_address,longitude,latitude,location,geometry
0,2018-347660,7660504950,2018-01-01,09/17/2018 02:08:00 PM,09/17/2018 04:05:00 PM,A,PROPERTY,LARCENY-THEFT,Theft From Building,23D,N,J,J1,BALLARD NORTH,8XX BLOCK OF NW 95TH ST,-122.366706,47.697866,"(47.69786645, -122.36670644)",POINT (-122.36671 47.69787)
1,2018-306159,7648277122,2018-01-01,08/17/2018 07:34:00 AM,08/17/2018 07:34:00 AM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,S,S,S2,RAINIER BEACH,86XX BLOCK OF ISLAND DR S,-122.261000,47.525617,"(47.52561712, -122.260999956)",POINT (-122.26100 47.52562)
3,2018-907128,7697846930,2018-01-01,01/01/2018 12:00:00 AM,08/04/2018 06:35:00 PM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,N,J,J3,PHINNEY RIDGE,7XX BLOCK OF N 70TH ST,-122.348588,47.679599,"(47.67959898, -122.348587683)",POINT (-122.34859 47.67960)
5,2018-214446,7678773428,2018-01-01,06/13/2018 02:41:00 PM,06/13/2018 02:41:00 PM,A,PROPERTY,FRAUD OFFENSES,False Pretenses/Swindle/Confidence Game,26A,S,O,O2,SODO,8XX BLOCK OF S ADAMS ST,-122.321761,47.566196,"(47.56619631, -122.321760505)",POINT (-122.32176 47.56620)
6,2018-137495,7700168935,2018-01-01,04/19/2018 09:53:00 AM,04/19/2018 09:53:00 AM,A,PROPERTY,COUNTERFEITING/FORGERY,Counterfeiting/Forgery,250,S,O,O2,SODO,38XX BLOCK OF 1ST AVE S,-122.334206,47.569889,"(47.56988892, -122.334205833)",POINT (-122.33421 47.56989)


In [28]:
gdf.set_crs(epsg=4326, inplace=True)
zipcode.set_crs(epsg=4326, inplace=True)
result = gpd.sjoin(gdf, zipcode, how="left", op='within')

/Users/yujinlee/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [29]:
result.head()

,report_number,offense_id,offense_start_datetime,offense_end_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,...,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10
0,2018-347660,7660504950,2018-01-01,09/17/2018 02:08:00 PM,09/17/2018 04:05:00 PM,A,PROPERTY,LARCENY-THEFT,Theft From Building,23D,...,98117,5398117,B5,G6350,S,10214490.0,1569166.0,+47.6877607,-122.3848701,N
1,2018-306159,7648277122,2018-01-01,08/17/2018 07:34:00 AM,08/17/2018 07:34:00 AM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,...,98118,5398118,B5,G6350,S,16178230.0,6840313.0,+47.5402459,-122.2685446,N
3,2018-907128,7697846930,2018-01-01,01/01/2018 12:00:00 AM,08/04/2018 06:35:00 PM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,...,98103,5398103,B5,G6350,S,12005476.0,1705646.0,+47.6782553,-122.3381521,N
5,2018-214446,7678773428,2018-01-01,06/13/2018 02:41:00 PM,06/13/2018 02:41:00 PM,A,PROPERTY,FRAUD OFFENSES,False Pretenses/Swindle/Confidence Game,26A,...,98108,5398108,B5,G6350,S,19384665.0,964749.0,+47.5348617,-122.3057936,N
6,2018-137495,7700168935,2018-01-01,04/19/2018 09:53:00 AM,04/19/2018 09:53:00 AM,A,PROPERTY,COUNTERFEITING/FORGERY,Counterfeiting/Forgery,250,...,98134,5398134,B5,G6350,S,8738521.0,1888144.0,+47.5775985,-122.3380449,N


In [30]:
result.set_index('offense_start_datetime', inplace=True)

In [31]:
result.index = pd.to_datetime(result.index)
result.head()

,report_number,offense_id,offense_end_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,precinct,...,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10
offense_start_datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01,2018-347660,7660504950,09/17/2018 02:08:00 PM,09/17/2018 04:05:00 PM,A,PROPERTY,LARCENY-THEFT,Theft From Building,23D,N,...,98117,5398117,B5,G6350,S,10214490.0,1569166.0,+47.6877607,-122.3848701,N
2018-01-01,2018-306159,7648277122,08/17/2018 07:34:00 AM,08/17/2018 07:34:00 AM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,S,...,98118,5398118,B5,G6350,S,16178230.0,6840313.0,+47.5402459,-122.2685446,N
2018-01-01,2018-907128,7697846930,01/01/2018 12:00:00 AM,08/04/2018 06:35:00 PM,A,PROPERTY,FRAUD OFFENSES,Impersonation,26C,N,...,98103,5398103,B5,G6350,S,12005476.0,1705646.0,+47.6782553,-122.3381521,N
2018-01-01,2018-214446,7678773428,06/13/2018 02:41:00 PM,06/13/2018 02:41:00 PM,A,PROPERTY,FRAUD OFFENSES,False Pretenses/Swindle/Confidence Game,26A,S,...,98108,5398108,B5,G6350,S,19384665.0,964749.0,+47.5348617,-122.3057936,N
2018-01-01,2018-137495,7700168935,04/19/2018 09:53:00 AM,04/19/2018 09:53:00 AM,A,PROPERTY,COUNTERFEITING/FORGERY,Counterfeiting/Forgery,250,S,...,98134,5398134,B5,G6350,S,8738521.0,1888144.0,+47.5775985,-122.3380449,N


In [32]:
weekly_data = result.groupby('ZCTA5CE10').resample('W').size().reset_index(name='total_count')

In [33]:
weekly_data['ZCTA5CE10'].unique().shape

(34,)

In [34]:
weekly_data

,ZCTA5CE10,offense_start_datetime,total_count
0,98101,2018-01-07,126
1,98101,2018-01-14,132
2,98101,2018-01-21,127
3,98101,2018-01-28,110
4,98101,2018-02-04,141
...,...,...,...
9021,98199,2023-05-07,9
9022,98199,2023-05-14,10
9023,98199,2023-05-21,8
9024,98199,2023-05-28,13


In [36]:
mean_counts_by_zip = weekly_data.groupby('ZCTA5CE10')['total_count'].mean().reset_index()
mean_counts_by_zip

,ZCTA5CE10,total_count
0,98101,90.424028
1,98102,43.469965
2,98103,83.901060
3,98104,84.780919
4,98105,81.819788
5,98106,41.692580
6,98107,54.614841
7,98108,46.155477
8,98109,73.537102
9,98112,32.749117


In [37]:
weekly_data.to_csv(file_path + '/seattle_weekly_data.csv', index=False)